# Tensorflow ImageNet Resnet50 FGM demo

>⚠️ **Warning:** This demo assumes that you have access to an on-prem deployment of Dioptra that provides a copy of the ImageNet dataset and a CUDA-compatible GPU.
> This demo cannot be run on a typical personal computer.

The notebook contains an example of the FGM attack on the ResNet50 architecture with optional defense entry points.
Users can also explore the following preprocessing defenses from their associated defense entry points:

-   **Spatial Smoothing:** Smooths out an image by passing a median filter through neighboring pixel values in images.
-   **Gaussian Augmentation:** Adds gaussian noise to an image.
-   **JPEG Compression:** Applies an image compression algorithm over the image.

## Setup

**Note:** This demo is specifically for the NCCoE DGX Workstation with hostname `dgx-station-2`.

Port forwarding is required in order to run this demo.
The recommended port mapping is as follows:

- Map `*:20080` on laptop to `localhost:30080` on `dgx-station-2`
- Map `*:25000` on laptop to `localhost:35000` on `dgx-station-2`

A sample SSH config file that enables the above port forwarding is provided below,

> ⚠️ **Edits required**: replace `username` with your assigned username _on the NCCoE virtual machines_!

```conf
# vm hostname: jumphost001
Host nccoe-jumphost001
    Hostname 10.33.53.98
    User username  # Change to your assigned username on the NCCoE virtual machines!
    Port 54131
    IdentityFile %d/.ssh/nccoe-vm

# vm hostname: dgx-station-2
Host nccoe-k8s-gpu002
    Hostname 192.168.1.28
    User username  # Change to your assigned username on the NCCoE virtual machines!
    Port 22
    IdentityFile %d/.ssh/nccoe-vm
    ProxyJump nccoe-jumphost001
    LocalForward *:20080 localhost:30080
    LocalForward *:25000 localhost:35000
```

Now, connect to the NCCoE VPN and SSH into the DGX Workstation,

```bash
ssh nccoe-k8s-gpu002
```

Next, we import the necessary Python modules and ensure the proper environment variables are set so that all the code blocks will work as expected.

> ⚠️ **Edits possibly required**: update the value of the `HOST_DOCKER_INTERNAL` variable

If this notebook is being served to you via Docker (i.e. you ran `make jupyter` to launch this notebook), **then you may need to change the value assigned to the variable `HOST_DOCKER_INTERNAL`** to make the port forwarding you configured in the previous step accessible within the container.
The value you need to assign to the variable to depends on your host device's operating system:

- **Case 1: Host operating system is Windows 10 or MacOS**
  - Set `HOST_DOCKER_INTERNAL = "host.docker.internal"`. This is the default setting.
- **Case 2: Host operating system is Linux**
  - Run either `ip address` or `ifconfig` to print a list of the available network interfaces on your host device
  - Locate the `docker0` interface and take note of the associated IP address (this is commonly set to `172.17.0.1`)
  - Set `HOST_DOCKER_INTERNAL` equal to the IP address for the `docker0` interface. So, if the IP address was `172.17.0.1`, then you would set `HOST_DOCKER_INTERNAL = "172.17.0.1"`
  
If you started your Jupyter Lab instance from a conda environment, then you do not need to change anything.
The code below uses an environment variable to check whether this notebook is being served via the `jupyter` service, and if that variable isn't found, then the connection address reverts to `localhost` and ignores the `HOST_DOCKER_INTERNAL` variable.

In [ ]:
# Import packages from the Python standard library
import os
import pprint
import time
import warnings
from pathlib import Path
from typing import Tuple

# Please enter custom username here.
USERNAME = "howard"

# Filter out warning messages
warnings.filterwarnings("ignore")

# Address for connecting the docker container to exposed ports on the host device
HOST_DOCKER_INTERNAL = "host.docker.internal"
# HOST_DOCKER_INTERNAL = "172.17.0.1"

# Testbed API ports
RESTAPI_PORT = "30080"
MLFLOW_TRACKING_PORT = "35000"

# Default address for accessing the RESTful API service
RESTAPI_ADDRESS = (
    f"http://{HOST_DOCKER_INTERNAL}:{RESTAPI_PORT}"
    if os.getenv("IS_JUPYTER_SERVICE")
    else f"http://localhost:{RESTAPI_PORT}"
)

# Override the AI_RESTAPI_URI variable, used to connect to RESTful API service
os.environ["AI_RESTAPI_URI"] = RESTAPI_ADDRESS

# Default address for accessing the MLFlow Tracking server
MLFLOW_TRACKING_URI = (
    f"http://{HOST_DOCKER_INTERNAL}:{MLFLOW_TRACKING_PORT}"
    if os.getenv("IS_JUPYTER_SERVICE")
    else f"http://localhost:{MLFLOW_TRACKING_PORT}"
)

# Override the MLFLOW_TRACKING_URI variable, used to connect to MLFlow Tracking service
os.environ["MLFLOW_TRACKING_URI"] = MLFLOW_TRACKING_URI

# Path to custom task plugins archives
CUSTOM_PLUGINS_EVALUATION_TAR_GZ = Path("custom-plugins-evaluation.tar.gz")

# Base API address
RESTAPI_API_BASE = f"{RESTAPI_ADDRESS}/api"

# Path to workflows archive
WORKFLOWS_TAR_GZ = Path("workflows.tar.gz")

# Experiment name (note the username_ prefix convention)
EXPERIMENT_NAME = f"{USERNAME}_imagenet_fgm_defense"

# Path to dataset
data_path_imagenet = "/nfs/data/ImageNet-Kaggle-2017/images/ILSVRC/Data/CLS-LOC"

# Import third-party Python packages
import numpy as np
import requests
from mlflow.tracking import MlflowClient

# Import utils.py file
import utils

# Create random number generator
rng = np.random.default_rng(54399264723942495723666216079516778448)

## Dataset

The ImageNet dataset needed for the demo is available in all worker containers at the path `/nfs/data/ImageNet-Kaggle-2017/images/ILSVRC/Data/CLS-LOC/`.
The following directores contain labeled JPEG images:

    train  
    val-sorted  
    val-sorted-1000
    val-sorted-5000
    val-sorted-10000  
    
Both `train` and `val-sorted` contain a full set of labeled training and test images.
The `val-sorted-#` directories hold a random partion of labeled images, with 1000, 5000, or 10000 images in total. 

Each of the directories listed follows the general subfolder structure below:

    ImageNet
    ├── train
    │   ├── {class 0 directory}
    │   ├── {class 1 directory}
    │   ├── ...
    │   ├── ...
    │   ├── ...
    │   └── {class 999 directory}
    ├── val-sorted
    │   ├── {class 0 directory}
    │   ├── {class 1 directory}
    │   ├── ...
    │   ├── ...
    │   ├── ...
    │   └── {class 999 directory}
    ...

The subfolders under each directory are the classification labels for the images in the dataset.
This folder structure is a standardized way to encode the label information and many libraries can make use of it, including the Tensorflow library that we are using for this particular demo.

## Submit and run jobs

The entrypoints that we will be running in this example are implemented in the Python source files under `src/` and the `MLproject` file.
To run these entrypoints within the testbed architecture, we need to package those files up into an archive and submit it to the Testbed RESTful API to create a new job.
For convenience, the `Makefile` provides a rule for creating the archive file for this example, just run `make workflows`,

In [ ]:
%%bash

# Create the workflows.tar.gz file
make workflows

To connect with the endpoint, we will use a client class defined in the `utils.py` file that is able to connect with the Testbed RESTful API using the HTTP protocol.
We connect using the client below, which uses the environment variable `AI_RESTAPI_URI` to figure out how to connect to the Testbed RESTful API,

In [ ]:
restapi_client = utils.SecuringAIClient()

We need to register an experiment under which to collect our job runs.
The code below checks if the relevant experiment exists.
If it does, then it just returns info about the experiment, if it doesn't, it then registers the new experiment.

In [ ]:
response_experiment = restapi_client.get_experiment_by_name(name=EXPERIMENT_NAME)

if response_experiment is None or "Not Found" in response_experiment.get("message", []):
    response_experiment = restapi_client.register_experiment(name=EXPERIMENT_NAME)

response_experiment

We should also check which queues are available for running our jobs to make sure that the resources that we need are available.
The code below queries the Testbed API and returns a list of active queues.

In [ ]:
restapi_client.list_queues()

In [ ]:
response_queue = restapi_client.get_queue_by_name(name="tensorflow_cpu")

if response_queue is None or "Not Found" in response_queue.get("message", []):
    response_queue = restapi_client.register_queue(name="tensorflow_cpu")

response_queue

This example also makes use of the `custom_fgm_patch_poisoning_plugins` custom task plugin package stored locally under the `task-plugins/securingai_custom/custom_fgm_patch_poisoning_plugins` directory.
To register these custom task plugins, we first need to package them up into an archive.
For convenience, the `Makefile` provides a rule for creating the custom task plugins archive file, just run `make custom-plugins`,

In [ ]:
%%bash

# Create the workflows.tar.gz file
make custom-plugins

Now that the custom task plugin package is packaged into an archive file, next we register it by uploading the file to the REST API.
Note that we need to provide the name to use for custom task plugin package, this name must be unique under the custom task plugins namespace.
For a full list of the custom task plugins, use `restapi_client.restapi_client.list_custom_task_plugins()`.

In [ ]:
restapi_client.delete_custom_task_plugin(name="custom_fgm_patch_poisoning_plugins")
response_custom_plugins = restapi_client.get_custom_task_plugin(name="custom_fgm_patch_poisoning_plugins")

if response_custom_plugins is None or "Not Found" in response_custom_plugins.get("message", []):
    response_custom_plugins = restapi_client.upload_custom_plugin_package(
        custom_plugin_name="custom_fgm_patch_poisoning_plugins",
        custom_plugin_file=CUSTOM_PLUGINS_EVALUATION_TAR_GZ,
    )

response_custom_plugins

If at any point you need to update one or more files within the `evaluation` plugin package, you will need to unregister/delete the custom task plugin first using the REST API.
This can be done as follows,

```python
# Delete the 'evaluation' custom task plugin package
restapi_client.delete_custom_task_plugin(name="custom_fgm_patch_poisoning_plugins")
```

# FGM Model Setup and Attack

For this section of the demo we will first start by loading in the respective baseline ResNet50 model and evaluating its performance on a subset of the ImageNet validation data.

In [ ]:
response_init_model = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="init_model",
    entry_point_kwargs=" ".join([
        "-P batch_size=20",
        "-P model_architecture=resnet50",
        f"-P register_model_name={EXPERIMENT_NAME}_pretrained_resnet50",
        "-P imagenet_preprocessing=true",
        "-P image_size=224,224,3",
        f"-P data_dir={data_path_imagenet}/val-sorted-1000",
    ]),
    queue="tensorflow_gpu",
    timeout="1h",
)

print("Model initialization and ImageNet evaluation job submitted.")
print("")
pprint.pprint(response_init_model)

Now that we have our baseline model loaded and stored, next we will apply the fast-gradient method (FGM) evasion attack on the model to generate adversarial images.
Then, after we have the adversarial images, we will reapply the attack with spatial smoothing defense enabled.
We will then test the model under both sets of images (FGM attack, FGM attack + preprocessing defense) and apply standard machine learning accuracy metrics on the inference results. 
This will give us a general sense of the effectivness and robustness of the FGM evasion attack with regards to a particular preprocessing defense.

In [ ]:
def mlflow_run_id_is_not_known(response):
    return response["mlflowRunId"] is None and response["status"] not in [
        "failed",
        "finished",
    ]

response_fgm_resnet50_attack = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="fgm",
    entry_point_kwargs=" ".join(
        [
            f"-P model_name={EXPERIMENT_NAME}_pretrained_resnet50",
            "-P model_version=None",
            f"-P data_dir={data_path_imagenet}/val-sorted-1000",
            "-P batch_size=20",
            "-P eps_step=0.1",
            "-P imagenet_preprocessing=true",
            "-P image_size=224,224,3",
            "-P eps=10"
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_init_model['jobId']
)

print("FGM attack (ResNet50 architecture) job submitted")
print("")
pprint.pprint(response_fgm_resnet50_attack)
print("")

# Wait for FGM attack to complete.Then evaluate results.
while mlflow_run_id_is_not_known(response_fgm_resnet50_attack):
    time.sleep(1)
    response_fgm_resnet50_attack = restapi_client.get_job_by_id(response_fgm_resnet50_attack["jobId"])

In [ ]:
response_evaluate_fgm_resnet50_attack = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="infer",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_fgm_resnet50_attack['mlflowRunId']}",
            f"-P model_name={EXPERIMENT_NAME}_pretrained_resnet50",
            "-P model_version=None",
            "-P batch_size=20",
            "-P adv_tar_name=testing_adversarial_fgm.tar.gz",
            "-P adv_data_dir=adv_testing",
            "-P imagenet_preprocessing=true",
            "-P image_size=224,224,3",
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_fgm_resnet50_attack["jobId"],
)

print("FGM evaluation (ResNet50 architecture) job submitted")
print("")
pprint.pprint(response_evaluate_fgm_resnet50_attack)
print("")

# Image Preprocessing Defenses
Now we will apply our defenses over the fgm-attacked images.

In [ ]:
response_fgm_resnet50_attack_def = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="spatial_smoothing",
    entry_point_kwargs=" ".join(
        [
            f"-P data_dir={data_path_imagenet}/val-sorted-1000",
            "-P batch_size=20",
            "-P load_dataset_from_mlruns=true",
            f"-P dataset_run_id={response_fgm_resnet50_attack['mlflowRunId']}",
            "-P dataset_tar_name=testing_adversarial_fgm.tar.gz",
            "-P dataset_name=adv_testing",
            "-P image_size=224,224,3",
            
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_fgm_resnet50_attack["jobId"],
)

print("FGM spatial smoothing defense (ResNet50 architecture) job submitted")
print("")
pprint.pprint(response_fgm_resnet50_attack_def)
print("")

In [ ]:
while mlflow_run_id_is_not_known(response_fgm_resnet50_attack_def):
    time.sleep(1)
    response_fgm_resnet50_attack_def = restapi_client.get_job_by_id(response_fgm_resnet50_attack_def["jobId"])
    
    
response_evaluate_fgm_resnet50_attack_def = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="infer",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_fgm_resnet50_attack_def['mlflowRunId']}",
            f"-P model_name={EXPERIMENT_NAME}_pretrained_resnet50",
            "-P model_version=None",
            "-P batch_size=20",
            "-P adv_tar_name=spatial_smoothing_dataset.tar.gz",
            "-P adv_data_dir=adv_testing",
            "-P imagenet_preprocessing=true",
            "-P image_size=224,224,3",
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_fgm_resnet50_attack_def["jobId"],
)

print("FGM evaluation (ResNet50 architecture) job submitted")
print("")
pprint.pprint(response_evaluate_fgm_resnet50_attack_def)
print("")

In [ ]:
response_fgm_resnet50_attack_def = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="gaussian_augmentation",
    entry_point_kwargs=" ".join(
        [
            f"-P data_dir={data_path_imagenet}/val-sorted-1000",
            "-P batch_size=20",
            "-P load_dataset_from_mlruns=true",
            f"-P dataset_run_id={response_fgm_resnet50_attack['mlflowRunId']}",
            "-P dataset_tar_name=testing_adversarial_fgm.tar.gz",
            "-P dataset_name=adv_testing",
            "-P image_size=224,224,3",
            
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_fgm_resnet50_attack["jobId"],
)

print("FGM gaussian augmentation defense (ResNet50 architecture) job submitted")
print("")
pprint.pprint(response_fgm_resnet50_attack_def)
print("")

In [ ]:
while mlflow_run_id_is_not_known(response_fgm_resnet50_attack_def):
    time.sleep(1)
    response_fgm_resnet50_attack_def = restapi_client.get_job_by_id(response_fgm_resnet50_attack_def["jobId"])
    
    
response_evaluate_fgm_resnet50_attack_def = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="infer",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_fgm_resnet50_attack_def['mlflowRunId']}",
            f"-P model_name={EXPERIMENT_NAME}_pretrained_resnet50",
            "-P model_version=None",
            "-P batch_size=20",
            "-P adv_tar_name=gaussian_augmentation_dataset.tar.gz",
            "-P adv_data_dir=adv_testing",
            "-P imagenet_preprocessing=true",
            "-P image_size=224,224,3",
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_fgm_resnet50_attack_def["jobId"],
)

print("FGM evaluation (ResNet50 architecture) job submitted")
print("")
pprint.pprint(response_evaluate_fgm_resnet50_attack_def)
print("")

In [ ]:
response_fgm_resnet50_attack_def = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="jpeg_compression",
    entry_point_kwargs=" ".join(
        [
            f"-P data_dir={data_path_imagenet}/val-sorted-1000",
            "-P batch_size=20",
            "-P load_dataset_from_mlruns=true",
            f"-P dataset_run_id={response_fgm_resnet50_attack['mlflowRunId']}",
            "-P dataset_tar_name=testing_adversarial_fgm.tar.gz",
            "-P dataset_name=adv_testing",
            "-P image_size=224,224,3",
            
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_fgm_resnet50_attack["jobId"],
)

print("FGM JPEG compression defense (ResNet50 architecture) job submitted")
print("")
pprint.pprint(response_fgm_resnet50_attack_def)
print("")

In [ ]:
while mlflow_run_id_is_not_known(response_fgm_resnet50_attack_def):
    time.sleep(1)
    response_fgm_resnet50_attack_def = restapi_client.get_job_by_id(response_fgm_resnet50_attack_def["jobId"])
    
    
response_evaluate_fgm_resnet50_attack_def = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="infer",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_fgm_resnet50_attack_def['mlflowRunId']}",
            f"-P model_name={EXPERIMENT_NAME}_pretrained_resnet50",
            "-P model_version=None",
            "-P batch_size=20",
            "-P adv_tar_name=jpeg_compression_dataset.tar.gz",
            "-P adv_data_dir=adv_testing",
            "-P imagenet_preprocessing=true",
            "-P image_size=224,224,3",
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_fgm_resnet50_attack_def["jobId"],
)

print("FGM evaluation (ResNet50 architecture) job submitted")
print("")
pprint.pprint(response_evaluate_fgm_resnet50_attack_def)
print("")